## Simple tests for grabbing frames

OpenCV has a nice wrapper around webcams. Let's test it there to get the simplest code that we would need in our library.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)
# define plots to be inserted interactively
import matplotlib.pyplot as plt
import time
import cv2
cv2.__version__

In [ ]:
class Source:
    def __init__(self, w=640, h=480):
        self.h, self.w = h, w
        import cv2
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.w)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.h)
      
    def grab(self):
        # grab a frame
        returned, cam_data = self.cap.read()
        data = cam_data.reshape((self.h, self.w, 3))
        return data
    
    def close(self):
        self.cap.release()
        del self.cap

start = time.time()
cam = Source()
cam.close()
print('Time to start and stop the camera = ',  time.time() - start , '(s)')


This is implemented in the ``LeCheapEyeTracker``class:

In [ ]:
from openRetina import PhotoReceptor

cam = PhotoReceptor()
cam.close()


In [ ]:
for DOWNSCALE in [1, 2, 4, 8, 16]:
    start = time.time()
    cam = PhotoReceptor(DOWNSCALE=DOWNSCALE)
    cam.close()
    print('DOWNSCALE = ', DOWNSCALE, 'startup= ',  time.time() - start , '(s)')

In [ ]:
start = time.time()
cam = PhotoReceptor(DOWNSCALE=2)
img = cam.grab()
img = cam.grab()
cam.close()
print('Time to start, grab 2 frames and stop = ',  time.time() - start , '(s)')
print(img.shape, img.min(), img.max())
_ = plt.imshow(img)

In [ ]:
for DOWNSCALE in [1, 2, 4, 8, 16]:
    cam = PhotoReceptor(DOWNSCALE=DOWNSCALE)
    start = time.time()
    img = cam.grab()
    img = cam.grab()
    cam.close()
    print('DOWNSCALE = ', DOWNSCALE, 'grab time= ',  time.time() - start , '(s)')

## how fast do we grab frames?

In [ ]:
N = 100
ctime = np.zeros(N)
start = time.time()
cam = PhotoReceptor()
for i in range(N):
    img = cam.grab()
    ctime[i] = time.time() - start
cam.close()

In [ ]:
plt.plot(ctime[1:], np.diff(ctime)*1000, '+')
plt.xlabel('time (s)')
plt.ylabel('inter-frame interval(ms)')
_ = plt.axis('tight')
print ('FPS : ',  N/(ctime[-1]-ctime[0]))

## closing the camera

In [ ]:
cam = PhotoReceptor()
cam.close()

## threaded mode

Following the documentation @ http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html?highlight=imread it is easy to run parallel captures :

In [ ]:
from multiprocessing.pool import ThreadPool
from collections import deque
import cv2
import time

class StatValue:
    def __init__(self, smooth_coef = 0.5):
        self.value = None
        self.smooth_coef = smooth_coef
    def update(self, v):
        if self.value is None:
            self.value = v
        else:
            c = self.smooth_coef
            self.value = c * self.value + (1.0-c) * v

def clock():
    return cv2.getTickCount() / cv2.getTickFrequency()

def draw_str(dst, target, s):
    x, y = target
    cv2.putText(dst, s, (x+1, y+1), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 0, 0), thickness = 2, lineType=cv2.LINE_AA)
    cv2.putText(dst, s, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)


class ThreadSource:
    def __init__(self, w=640, h=480, threads=True):
        self.h, self.w = h, w
        import cv2
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.w)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.h)

        if threads:
            self.threadn = cv2.getNumberOfCPUs()
        else:
            self.threadn = 1
        self.pool = ThreadPool(processes = self.threadn)
        self.pending = deque()

        self.latency = StatValue()
        self.frame_interval = StatValue()
        self.last_frame_time = clock()        
        self.display = False
        self.ctime = []
        self.N = 0

    def process_frame(self, frame, t0):
        # some intensive computation...
        frame = cv2.medianBlur(frame, 19)
        time.sleep(.1)
        frame = cv2.medianBlur(frame, 19)
        return frame, t0

    def run(self, T=10):
        start = clock()
        while clock()-start <10.:
            while len(self.pending) > 0 and self.pending[0].ready():
                res, t0 = self.pending.popleft().get()
                self.latency.update(clock() - t0)
                if self.display:
                    draw_str(res, (20, 40), "latency        :  %.1f ms" % (self.latency.value*1000))
                    draw_str(res, (20, 60), "frame interval :  %.1f ms" % (self.frame_interval.value*1000))
                    cv2.imshow('Webcam video', res)
                self.ctime.append(time.time() - start)
                self.N += 1
            if len(self.pending) < self.threadn:
                ret, frame = self.cap.read()
                t = clock()
                self.frame_interval.update(t - self.last_frame_time)
                self.last_frame_time = t
                task = self.pool.apply_async(self.process_frame, (frame.copy(), t))
                self.pending.append(task)
            ch = 0xFF & cv2.waitKey(1)
            if ch == 27:
                self.close()
    
    def close(self):
        self.cap.release()
        if self.display: cv2.destroyAllWindows()

start = time.time()
cam = ThreadSource()
ctime = cam.run()
cam.close()
start_nothreads = time.time()
cam_nothreads = ThreadSource(threads=False)
ctime = cam_nothreads.run()
cam_nothreads.close()

In [ ]:
plt.plot(np.array(cam.ctime)[1:]-cam.ctime[0], np.diff(np.array(cam.ctime))*1000, 'b+')
plt.plot(np.array(cam_nothreads.ctime)[1:]-cam_nothreads.ctime[0], np.diff(np.array(cam_nothreads.ctime))*1000, 'rx')
plt.xlabel('time (s)')
plt.ylabel('inter-frame interval(ms)')
_ = plt.axis('tight')
print ('FPS threaded: ',  cam.N/(cam.ctime[-1]-cam.ctime[0]))
print ('FPS one thread: ',  cam_nothreads.N/(cam_nothreads.ctime[-1]-cam_nothreads.ctime[0]))

In practice, we will see nice improvments when the image processing pipeline is more consequent. This threaded mode is enabled by default.

##  creating a video for debugging

It may be sometimes useful to just read out some frames for debugging: move your eyes, not the head = we grab 42 frames. why 42? I recommend to not move the head (hold them in your hands) and to look at the led of your webcam. when it's lit, move your eyes evenly on the screen.

In [1]:
import os
import time
import cv2

N_frame = 42 # how much you want?
folder = '/tmp/debug' # where we do save the frames
try: 
    os.mkdir(folder)
except: pass

from openRetina import PhotoReceptor
cam = PhotoReceptor()

time.sleep(1.5)

start = time.time()
for i in range(N_frame):
    frame = cam.grab()
    t = time.time() - start   
    # https://docs.python.org/3.3/library/string.html#format-examples
    timestr = '{t:03.5f}'.format(t=t).replace('.', '_')
    fname = 'frame_{i}.png'.format(i=i)
    print(fname)
    cv2.imwrite(os.path.join(folder, fname), frame)
    time.sleep(0.3)

cam.close()

frame_0.png
frame_1.png
frame_2.png
frame_3.png
frame_4.png
frame_5.png
frame_6.png
frame_7.png
frame_8.png
frame_9.png
frame_10.png
frame_11.png
frame_12.png
frame_13.png
frame_14.png
frame_15.png
frame_16.png
frame_17.png
frame_18.png
frame_19.png
frame_20.png
frame_21.png
frame_22.png
frame_23.png
frame_24.png
frame_25.png
frame_26.png
frame_27.png
frame_28.png
frame_29.png
frame_30.png
frame_31.png
frame_32.png
frame_33.png
frame_34.png
frame_35.png
frame_36.png
frame_37.png
frame_38.png
frame_39.png
frame_40.png
frame_41.png


## using the class in LecheapEyeTracker



In [ ]:
cam = PhotoReceptor()
cam.run(T=8)
cam.close()
if len(cam.ctime) > 0:
    plt.plot(np.diff(np.array(cam.ctime))*1000, '+')
    _ = plt.axis('tight')
    print ('FPS : ',  len(cam.ctime)/(cam.ctime[-1]-cam.ctime[0]))